In [1]:
import tensorflow as tf
eff = tf.keras.models.load_model('/Volumes/Janrauhl/model_weights/effnet_wo_tl.h5')
effb0 = tf.saved_model.load('/Volumes/Janrauhl/model_weights/effnet_b0_tl')

2023-08-28 12:49:29.010321: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2023-08-28 12:49:29.010338: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-08-28 12:49:29.010345: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-08-28 12:49:29.010373: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-28 12:49:29.010389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:

res = tf.keras.models.load_model('/Volumes/Janrauhl/model_weights/res_tl.h5')

In [3]:
from tensorflow.keras.layers import Input, Concatenate, Dense, GlobalAveragePooling2D 
from tensorflow.keras.models import Model
batch_size = 32
img_height = 200 
img_width = 200 
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'DownFER/train', # directory path
    validation_split = None, 
    subset = None, 
    image_size = (img_height, img_width), # image size of height and width
    batch_size = 32# batch size
)
testing_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'DownFER/test', # directory path
    validation_split = None, 
    subset = None, 
    image_size = (img_height, img_width), # image size of height and width
    batch_size = 32 # batch size
)
class_names = training_ds.class_names
print(class_names)
print(len(class_names))
# configuring dataset for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

# cache() keeps images in memory to ensure dataset does not become a bottleneck 
  # while training the model
# prefetch() overlaps data preprocessing and model execution while training
num_classes = len(class_names)
training_ds = training_ds.cache().prefetch(buffer_size = AUTOTUNE) # for training
testing_ds = testing_ds.cache().prefetch(buffer_size = AUTOTUNE) # for testing
# One-hot encode the labels
def one_hot_encode(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label

# Apply one-hot encoding to the dataset
training_ds = training_ds.map(one_hot_encode)
testing_ds = testing_ds.map(one_hot_encode)
from tensorflow.keras.applications.efficientnet import preprocess_input 
target_size =  (224,224) 

def preprocess_image(image,label): 
    image = tf.image.resize(image, target_size) 
    image = preprocess_input(image) 
    return image, label 
training_ds_resized = training_ds.map(preprocess_image) 
testing_ds_resized = testing_ds.map(preprocess_image) 
sample_image, _ = next(iter(testing_ds_resized.take(1)))
sample_image_shape = sample_image.shape

print("Shape of a sample image in training_ds_resized:", sample_image_shape)

def plot_model(model_saved, N, path = None): 
    import matplotlib.pyplot as plt
    import numpy as np
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), model_saved.history['loss'], 
             label = 'training loss') # training loss
    plt.plot(np.arange(0, N), model_saved.history['val_loss'], 
             label = 'test loss') # testing loss
    plt.plot(np.arange(0, N), model_saved.history['accuracy'], 
             label = 'training accuracy') # training accuracy
    plt.plot(np.arange(0, N), model_saved.history['val_accuracy'], 
             label = 'test accuracy') # testing accuracy
    plt.title("Training and Testing Loss and Accuracy") # title of comparative line graph
    plt.xlabel("Epoch #") # x axis label
    plt.ylabel("Loss/Accuracy") # y axis label
    plt.legend(loc = "upper right") # legend

    if path: 
        plt.savefig(path) 

    plt.show()


Found 26880 files belonging to 6 classes.
Found 7178 files belonging to 7 classes.
['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
6
Shape of a sample image in training_ds_resized: (32, 224, 224, 3)


2023-08-28 12:49:39.388217: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [4]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import layers
# batch_size = 32
# img_height = 224
# img_width = 224 

# inputs = layers.Input(shape=(img_width,img_height,3), batch_size = batch_size) 

# #using model without transfer learnaing
# outputs = EfficientNetB0(include_top = True, weights = None, classes = num_classes)(inputs) 
# eff_wo_tl = tf.keras.Model(inputs,outputs) 
# eff_wo_tl.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
# eff_wo_tl.summary() 
# eff_wo_tl.fit(training_ds_resized, validation_data = testing_ds_resized, batch_size = batch_size, epochs = 20) 




#efficient net b0 with transfer learning from image net
batch_size = 32
img_height = 224
img_width = 224 

inputs = layers.Input(shape=(img_height, img_width, 3))
x = inputs
eff_b0_tl = EfficientNetB0(include_top=False, input_tensor=x, weights=None)

# Freeze the pretrained weights
eff_b0_tl.trainable = False

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(eff_b0_tl.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

# Compile
eff_b0_tl = tf.keras.Model(inputs, outputs, name="EfficientNet")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
eff_b0_tl.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
eff_b0_tl.fit(training_ds_resized, validation_data = testing_ds_resized, batch_size = batch_size, epochs = 20) 

Epoch 1/20


2023-08-28 12:49:41.511254: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


840/840 [==============================] - ETA: 0s - loss: 1.8101 - accuracy: 0.1678

2023-08-28 12:50:19.642404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


840/840 [==============================] - 50s 57ms/step - loss: 1.8101 - accuracy: 0.1678 - val_loss: 1.6639 - val_accuracy: 0.0155
Epoch 2/20
840/840 [==============================] - 49s 58ms/step - loss: 1.7985 - accuracy: 0.1824 - val_loss: 1.6275 - val_accuracy: 0.1220
Epoch 3/20
840/840 [==============================] - 50s 59ms/step - loss: 1.7611 - accuracy: 0.2173 - val_loss: 1.6322 - val_accuracy: 0.1357
Epoch 4/20
840/840 [==============================] - 51s 61ms/step - loss: 1.7340 - accuracy: 0.2416 - val_loss: 1.6151 - val_accuracy: 0.1594
Epoch 5/20
840/840 [==============================] - 52s 61ms/step - loss: 1.7167 - accuracy: 0.2540 - val_loss: 1.5977 - val_accuracy: 0.1694
Epoch 6/20
840/840 [==============================] - 50s 60ms/step - loss: 1.7020 - accuracy: 0.2645 - val_loss: 1.5978 - val_accuracy: 0.1690
Epoch 7/20
840/840 [==============================] - 49s 59ms/step - loss: 1.6878 - accuracy: 0.2715 - val_loss: 1.5976 - val_accuracy: 0.1659
Epo

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
image_height = 224
image_width = 224
model = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(image_height, image_width, 3)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training_ds_resized, validation_data = testing_ds_resized, batch_size = batch_size, epochs = 20) 

Epoch 1/20


2023-08-28 13:05:56.963368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


840/840 [==============================] - ETA: 0s - loss: 21.4045 - accuracy: 0.3023

2023-08-28 13:06:48.581465: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


840/840 [==============================] - 56s 66ms/step - loss: 21.4045 - accuracy: 0.3023 - val_loss: 24.1477 - val_accuracy: 0.0571
Epoch 2/20
840/840 [==============================] - 56s 67ms/step - loss: 10.4773 - accuracy: 0.3555 - val_loss: 13.1755 - val_accuracy: 0.2285
Epoch 3/20
840/840 [==============================] - 56s 66ms/step - loss: 10.2265 - accuracy: 0.3404 - val_loss: 23.1162 - val_accuracy: 0.1559
Epoch 4/20
840/840 [==============================] - 56s 66ms/step - loss: 11.2041 - accuracy: 0.3527 - val_loss: 9.3247 - val_accuracy: 0.2317
Epoch 5/20
840/840 [==============================] - 278s 332ms/step - loss: 10.8778 - accuracy: 0.3562 - val_loss: 11.1868 - val_accuracy: 0.2044
Epoch 6/20
840/840 [==============================] - 55s 66ms/step - loss: 9.7427 - accuracy: 0.3747 - val_loss: 13.2995 - val_accuracy: 0.1686
Epoch 7/20
840/840 [==============================] - 55s 65ms/step - loss: 10.2722 - accuracy: 0.3744 - val_loss: 17.0608 - val_accura

In [7]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

batch_size = 32
img_height = 224
img_width = 224 

inputs = layers.Input(shape=(img_width,img_height,3), batch_size = batch_size) 

#using model without transfer learnaing
outputs = EfficientNetB0(include_top = True, weights = None, classes = num_classes)(inputs) 
eff_wo_tl = tf.keras.Model(inputs,outputs) 
eff_wo_tl.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 
eff_wo_tl.summary() 
eff_wo_tl.fit(training_ds_resized, validation_data = testing_ds_resized, batch_size = batch_size, epochs = 20) 
eff_wo_tl.save('model_weights/effnet_wo_tl.h5')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(32, 224, 224, 3)]       0         
                                                                 
 efficientnetb0 (Functional  (None, 6)                 4057257   
 )                                                               
                                                                 
Total params: 4057257 (15.48 MB)
Trainable params: 4015234 (15.32 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________
Epoch 1/20


2023-08-28 21:19:24.793906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


840/840 [==============================] - ETA: 0s - loss: 1.7350 - accuracy: 0.3353

2023-08-28 21:21:59.615143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


840/840 [==============================] - 169s 192ms/step - loss: 1.7350 - accuracy: 0.3353 - val_loss: 1.5270 - val_accuracy: 0.2527
Epoch 2/20
840/840 [==============================] - 162s 192ms/step - loss: 1.2824 - accuracy: 0.4863 - val_loss: 1.4363 - val_accuracy: 0.3476
Epoch 3/20
840/840 [==============================] - 167s 198ms/step - loss: 1.0768 - accuracy: 0.5721 - val_loss: 1.4403 - val_accuracy: 0.3690
Epoch 4/20
840/840 [==============================] - 166s 197ms/step - loss: 0.9353 - accuracy: 0.6347 - val_loss: 1.5455 - val_accuracy: 0.3247
Epoch 5/20
840/840 [==============================] - 164s 194ms/step - loss: 0.8382 - accuracy: 0.6720 - val_loss: 1.7855 - val_accuracy: 0.3725
Epoch 6/20
840/840 [==============================] - 162s 192ms/step - loss: 0.7325 - accuracy: 0.7139 - val_loss: 1.8279 - val_accuracy: 0.3930
Epoch 7/20
840/840 [==============================] - 160s 189ms/step - loss: 0.6139 - accuracy: 0.7641 - val_loss: 2.2309 - val_accura

/opt/homebrew/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
num_classes= 6 
eff_out = eff.layers[-2].output
eff_out = GlobalAveragePooling2D()(eff_out)
res_out = res.layers[-2].output
res_out = GlobalAveragePooling2D()(res_out)
concatenated = Concatenate()([eff_out,res_out])
output = Dense(num_classes, activation = 'softmax')(concatenated)
ensemble = Model(inputs = [eff.input,res.input],outputs =output)
ensemble.compile(loss='categorical-crossentropy',optmizer='adam',metrics=['accuracy'])
ensemble.fit(training_ds_resized, validation_data = testing_ds_resized, batch_size = 32, epochs = 20)

ValueError: Input 0 of layer "global_average_pooling2d_1" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 1024)

In [18]:
eff.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(32, 224, 224, 3)]       0         
                                                                 
 efficientnetb0 (Functional  (None, 6)                 4057257   
 )                                                               
                                                                 
Total params: 4057257 (15.48 MB)
Trainable params: 4015234 (15.32 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________


In [19]:
res.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 6)                 6150      
                                                                 
Total params: 25692038 (98.01 MB)
Trainable params: 25638918 (97.80 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [27]:
# Print information about the loaded SavedModel
print("Loaded model signatures:", effb0.signatures)
print("Loaded model functions:", dir(effb0))

Loaded model signatures: _SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input_7) at 0x3205279D0>})
Loaded model functions: ['__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_trackable_child', '_add_variable_with_custom_getter', '_checkpoint_dependencies', '_default_save_signature', '_deferred_dependencies', '_delete_tracking', '_deserialization_dependencies', '_deserialize_from_proto', '_export_to_saved_model_graph', '_gather_saveables_for_checkpoint', '_handle_deferred_dependencies', '_lookup_dependency', '_maybe_initialize_trackable', '_name_based_attribute_restore', '_name_based_restores', '_no_dependency', '_object_identifier', '_p

In [28]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Concatenate, Dense, Reshape
from tensorflow.keras.models import Model

# Your code for creating 'eff' and 'res' models

num_classes = 6 

# Assuming you have 'eff' and 'res' models defined somewhere above
eff_out = eff.layers[-2].output
eff_out = GlobalAveragePooling2D()(eff_out)

res_out = res.layers[-2].output
res_out = GlobalAveragePooling2D()(res_out)
res_out = Reshape((1, 1, -1))(res_out)  # Reshape to (None, 1, 1, 2048)

# Concatenate the processed outputs
concatenated = Concatenate()([eff_out, res_out])

# Create the final output layer
output = Dense(num_classes, activation='softmax')(concatenated)

# Create the ensemble model
ensemble = Model(inputs=[eff.input, res.input], outputs=output)

ensemble.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Assuming 'training_ds_resized' and 'testing_ds_resized' are your training and testing datasets
ensemble.fit(training_ds_resized, validation_data=testing_ds_resized, batch_size=32, epochs=20)


AttributeError: '_UserObject' object has no attribute 'layers'